In [ ]:
!pip install keras_tuner
!pip install boto3
# !pip install --upgrade sqlalchemy
# !pip install --upgrade pandas
# !pip install sklearn-extensions
# !pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import csv

rds_host = ""
rds_port = ""
rds_username = ""
rds_password = ""
rds_database = ""

drive.mount('/content/drive')

with open('/content/drive/MyDrive/UofT/Data-Analytics-Boot-Camp/Project04-Team01/Keys/rds_Keys.csv') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    rds_host = row[0]
    rds_port = row[1]
    rds_username = row[2]
    rds_password = row[3]
    rds_database = row[4]

drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine(f'postgresql://{rds_username}:{rds_password}@{rds_host}:{rds_port}/{rds_database}')

with engine.begin() as connection:
  data = pd.read_sql(text('SELECT * FROM heart_disease'), con=connection)
  connection.close()

engine.dispose()
del engine

In [ ]:
feature_columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]
target_columns = ["target"]
numerical_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_columns = [x for x in feature_columns if x not in numerical_columns]

In [ ]:
data = data.loc[:, feature_columns + target_columns]

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib
# Scaling the numeric columns
scaler = StandardScaler().fit(data[feature_columns])
joblib.dump(scaler, "HeartScaler.sav")
data_scaled = scaler.transform(data[feature_columns])
data_scaled_df = pd.DataFrame(data_scaled, columns=feature_columns)
data_scaled_df = pd.concat([data_scaled_df, data[target_columns]], axis = 1)
data_scaled_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.936181,0.691095,-2.240629,0.750380,-0.276443,2.430427,1.010199,0.017494,-0.696419,1.068965,2.264145,-0.721976,0.655877,0
1,1.378929,0.691095,0.873880,1.596266,0.744555,-0.411450,1.010199,-1.816334,1.435916,0.381773,0.643781,2.478425,-0.894220,1
2,1.378929,0.691095,0.873880,-0.659431,-0.353500,-0.411450,1.010199,-0.899420,1.435916,1.326662,0.643781,1.411625,1.172577,1
3,-1.941680,0.691095,-0.164289,-0.095506,0.051047,-0.411450,-1.003419,1.633010,-0.696419,2.099753,2.264145,-0.721976,-0.894220,0
4,-1.498933,-1.446980,-1.202459,-0.095506,-0.835103,-0.411450,1.010199,0.978071,-0.696419,0.295874,-0.976583,-0.721976,-0.894220,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.272059,-1.446980,0.873880,0.468418,-0.122330,-0.411450,-1.003419,-1.161395,1.435916,-0.734914,0.643781,-0.721976,1.172577,1
293,-1.056185,0.691095,-2.240629,-1.223355,0.320744,-0.411450,-1.003419,-0.768432,-0.696419,0.124076,0.643781,-0.721976,1.172577,1
294,1.489615,0.691095,0.873880,0.693988,-1.047008,2.430427,-1.003419,-0.375469,-0.696419,2.013854,0.643781,1.411625,1.172577,1
295,0.272059,0.691095,0.873880,-0.095506,-2.241384,-0.411450,-1.003419,-1.510696,1.435916,0.124076,0.643781,0.344824,1.172577,1


In [ ]:
X = data_scaled_df[feature_columns]
y = data_scaled_df[target_columns]

In [ ]:
data_scaled_df[target_columns].value_counts()

target
0         160
1         137
dtype: int64

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X, y)

In [ ]:
# Skip Over Sampling
X_resampled = X
y_resampled = y
input_dim = len(X_resampled.columns)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(X_resampled,
                                                    y_resampled,
                                                    train_size=0.8,
                                                    random_state=12,
                                                    stratify=y_resampled)

Use "Logistic Regression" for supervised learning

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train.values.ravel())
predictions = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, predictions)

array([[31,  1],
       [ 7, 21]])

Classification report for "Logistic Regression"

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.82      0.97      0.89        32
      Absent       0.95      0.75      0.84        28

    accuracy                           0.87        60
   macro avg       0.89      0.86      0.86        60
weighted avg       0.88      0.87      0.86        60



Use "SVM" for supervised learning

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train.values.ravel())
predictions = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, predictions)

array([[31,  1],
       [ 7, 21]])

Classification report for "SVM"

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.82      0.97      0.89        32
      Absent       0.95      0.75      0.84        28

    accuracy                           0.87        60
   macro avg       0.89      0.86      0.86        60
weighted avg       0.88      0.87      0.86        60



Use "Random Forest" for supervised learning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, random_state=78)
model.fit(X_train, y_train.values.ravel())
predictions = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, predictions)

array([[29,  3],
       [ 7, 21]])

Classification report for "Random Forest"

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.81      0.91      0.85        32
      Absent       0.88      0.75      0.81        28

    accuracy                           0.83        60
   macro avg       0.84      0.83      0.83        60
weighted avg       0.84      0.83      0.83        60



Use "K Nearest Neighbors" for supervised learning

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train.values.ravel())
predictions = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, predictions)

array([[30,  2],
       [ 4, 24]])

Classification report for "K Nearest Neighbors"

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.88      0.94      0.91        32
      Absent       0.92      0.86      0.89        28

    accuracy                           0.90        60
   macro avg       0.90      0.90      0.90        60
weighted avg       0.90      0.90      0.90        60



In [ ]:
import pickle
pickle.dump(model, open("heart_model.sav", 'wb'))

Use "Decision Tree" for supervised learning

In [ ]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train.values.ravel())
predictions = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, predictions)

array([[26,  6],
       [ 6, 22]])

Classification report for "Decision Tree"

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.81      0.81      0.81        32
      Absent       0.79      0.79      0.79        28

    accuracy                           0.80        60
   macro avg       0.80      0.80      0.80        60
weighted avg       0.80      0.80      0.80        60



Use "Neural Networks" for supervised learning

In [ ]:
import tensorflow as tf

In [ ]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=50,
        step=5), activation=activation, input_dim=13))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=20,
            max_value=50,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

Use Keras tuner to find the best parameters for the neural network model

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    overwrite=True,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 180 Complete [00h 00m 07s]
val_accuracy: 0.8666666746139526

Best val_accuracy So Far: 0.8999999761581421
Total elapsed time: 00h 09m 13s


Parameters for the top 3 best neural network models

In [ ]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 30, 'num_layers': 3, 'units_0': 50, 'units_1': 40, 'units_2': 35, 'tuner/epochs': 17, 'tuner/initial_epoch': 6, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0060'}
{'activation': 'relu', 'first_units': 25, 'num_layers': 3, 'units_0': 25, 'units_1': 35, 'units_2': 25, 'tuner/epochs': 17, 'tuner/initial_epoch': 6, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0063'}
{'activation': 'relu', 'first_units': 25, 'num_layers': 3, 'units_0': 25, 'units_1': 35, 'units_2': 25, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0071'}


Accuracies of the top 3 best neural network models

In [ ]:
top_models = tuner.get_best_models(3)
for model in top_models:
    model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.3778 - accuracy: 0.9000 - 264ms/epoch - 132ms/step
Loss: 0.37784066796302795, Accuracy: 0.8999999761581421
2/2 - 0s - loss: 0.3981 - accuracy: 0.9000 - 273ms/epoch - 137ms/step
Loss: 0.39805153012275696, Accuracy: 0.8999999761581421
2/2 - 0s - loss: 0.3817 - accuracy: 0.9000 - 255ms/epoch - 128ms/step
Loss: 0.3817470371723175, Accuracy: 0.8999999761581421


In [ ]:
predictions = top_models[0].predict(X_test)
predictions = [int(round(float(prediction), 0)) for prediction in predictions]

2/2 [==============================] - 0s 6ms/step


In [ ]:
confusion_matrix(y_test, predictions)

array([[31,  1],
       [ 5, 23]])

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.86      0.97      0.91        32
      Absent       0.96      0.82      0.88        28

    accuracy                           0.90        60
   macro avg       0.91      0.90      0.90        60
weighted avg       0.91      0.90      0.90        60



In [ ]:
top_models[0].save("HeartDisease.h5")

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

top_param = top_hyper[0]

top_activation = top_param['activation']

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_param['first_units'], activation="relu", input_dim=input_dim))

# Add remaining hidden layers
for i in range(0, top_param['num_layers']):
  nn.add(tf.keras.layers.Dense(units=top_param[f'units_{i}'], activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 30)                420       
                                                                 
 dense_6 (Dense)             (None, 50)                1550      
                                                                 
 dense_7 (Dense)             (None, 40)                2040      
                                                                 
 dense_8 (Dense)             (None, 35)                1435      
                                                                 
 dense_9 (Dense)             (None, 1)                 36        
                                                                 
Total params: 5,481
Trainable params: 5,481
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.fit(X_resampled, y_resampled, initial_epoch=top_param["tuner/initial_epoch"], epochs=top_param['tuner/epochs'])

Epoch 7/17
10/10 [==============================] - 1s 4ms/step - loss: 0.6627 - accuracy: 0.6667
Epoch 8/17
10/10 [==============================] - 0s 2ms/step - loss: 0.5894 - accuracy: 0.7845
Epoch 9/17
10/10 [==============================] - 0s 2ms/step - loss: 0.5114 - accuracy: 0.8114
Epoch 10/17
10/10 [==============================] - 0s 2ms/step - loss: 0.4336 - accuracy: 0.8451
Epoch 11/17
10/10 [==============================] - 0s 2ms/step - loss: 0.3778 - accuracy: 0.8620
Epoch 12/17
10/10 [==============================] - 0s 2ms/step - loss: 0.3479 - accuracy: 0.8754
Epoch 13/17
10/10 [==============================] - 0s 2ms/step - loss: 0.3298 - accuracy: 0.8721
Epoch 14/17
10/10 [==============================] - 0s 2ms/step - loss: 0.3120 - accuracy: 0.8788
Epoch 15/17
10/10 [==============================] - 0s 2ms/step - loss: 0.2987 - accuracy: 0.8822
Epoch 16/17
10/10 [==============================] - 0s 3ms/step - loss: 0.2903 - accuracy: 0.8855
Epoch 17/17
1

In [ ]:
predictions = nn.predict(X_test)
predictions = [int(round(float(prediction), 0)) for prediction in predictions]

2/2 [==============================] - 0s 5ms/step


In [ ]:
confusion_matrix(y_test, predictions)

array([[31,  1],
       [ 5, 23]])

In [ ]:
print(classification_report(y_test, predictions, target_names=["Present", "Absent"]))

              precision    recall  f1-score   support

     Present       0.86      0.97      0.91        32
      Absent       0.96      0.82      0.88        28

    accuracy                           0.90        60
   macro avg       0.91      0.90      0.90        60
weighted avg       0.91      0.90      0.90        60



In [ ]:
# pickle.dump(nn, open("heart_model.sav", 'wb'))
nn.save("HeartDisease.h5")

In [ ]:
import json
with open('ConfusionMatrix.json', 'w') as file:
  file.write(json.dumps(confusion_matrix(y_test, predictions).tolist()))

In [ ]:
with open('ClassificationReport.json', 'w') as file:
  file.write(json.dumps(classification_report(y_test, predictions, target_names=["Present", "Absent"], output_dict=True)))

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.2763 - accuracy: 0.9000 - 182ms/epoch - 91ms/step
Loss: 0.2762989401817322, Accuracy: 0.8999999761581421


In [ ]:
from google.colab import drive
import csv

aws_access_key_id = ""
aws_secret_access_key = ""

drive.mount('/content/drive')

with open('/content/drive/MyDrive/UofT/Data-Analytics-Boot-Camp/Project04-Team01/Keys/collab_accessKeys.csv') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    aws_access_key_id = row[0]
    aws_secret_access_key=row[1]

drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
import boto3
session = boto3.session.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3 = session.resource('s3')
my_bucket = s3.Bucket('protected-06062023')
my_bucket.upload_file('HeartDisease.h5', 'HeartDisease.h5')
my_bucket.upload_file('HeartScaler.sav', 'HeartScaler.sav')
my_bucket.upload_file('heart_model.sav', 'heart_model.sav')
my_bucket.upload_file('ConfusionMatrix.json', 'ConfusionMatrix.json')
my_bucket.upload_file('ClassificationReport.json', 'ClassificationReport.json')